# Study Summaries Comparison

## Clustering

We are going to organize both summaries into predefined themes.  

By analyzing both summaries we define 4 themes for both summaries in such a way that the theme content is discussing roughly the same portion of the original study.


In [61]:
# MySummary.txt themes
my_themes = [
    'Introduction and Motivation',
    'Research Methodology',
    'Findings and Analysis',
    'Limitations and Future Work']

# LLM_Summary.txt themes
llm_themes = [
    'Introduction',
    'Methodology',
    'Key Findings',
    'Conclusion and Future Work']

We are defining a function that is splitting the summaries text into sections based on the theme headings:


In [62]:
import re

def extract_sections(text, themes):
    """
    Splits text into sections based on the provided theme headings.
    Returns a dictionary with theme as key and corresponding text as value.
    """
    sections = {}
    # Create a regex pattern that matches any of the theme headings.
    # (Assuming that headings appear at the beginning of a line)
    pattern = r'(?m)^(' + '|'.join(re.escape(theme) for theme in themes) + r')'
    
    # Find all matches and split text accordingly.
    splits = re.split(pattern, text)
    # re.split returns a list where headings are also part of the result.
    # The first element is any text before the first heading (if any).
    current_heading = None
    for segment in splits:
        segment = segment.strip()
        if segment in themes:
            current_heading = segment
            sections[current_heading] = ""
        elif current_heading:
            sections[current_heading] += segment + "\n"
    return sections


Reading the summary files:

In [63]:
# Read the files
with open("MySummary.txt", "r", encoding="utf-8") as file:
    my_summary_text = file.read()

with open("LLM_Summary.txt", "r", encoding="utf-8") as file:
    llm_summary_text = file.read()


Extracting the sections for each summary. The sections will be stored in a dictionary with keys from `my_themes` and `llm_themes` and the values will be the text content of the section.

In [64]:
# Extract sections for each summary
my_sections = extract_sections(my_summary_text, my_themes)
llm_sections = extract_sections(llm_summary_text, llm_themes)

Checking and validation the section split result:

In [65]:
# check the number of sections
print("my_sections length:", len(my_sections))
print("llm_sections length:", len(llm_sections))

my_sections length: 4
llm_sections length: 4


Visually inspecting the last section for both cases:

In [66]:
print(my_sections.keys())

dict_keys(['Introduction and Motivation', 'Research Methodology', 'Findings and Analysis', 'Limitations and Future Work'])


In [67]:
print(my_sections['Limitations and Future Work'])

While domain models can clearly highlight missing requirements, this study did not evaluate whether analysts effectively identify and correct those omissions in practice. Future research should include user studies to explore the practical effectiveness of domain models in supporting requirements validation.

Conclusion
This empirical study provides concrete evidence supporting domain models' value as effective tools for completeness checking in natural-language requirements specifications. By systematically highlighting omissions, particularly entirely missing requirements, domain models can significantly improve requirements quality, making them valuable components of requirements engineering practice.



In [68]:
print(llm_sections.keys())

dict_keys(['Introduction', 'Methodology', 'Key Findings', 'Conclusion and Future Work'])


In [69]:
print(llm_sections['Conclusion and Future Work'])

The study provides empirical evidence that domain models can help identify missing and under-specified requirements, though their effectiveness depends on how frequently concepts are referenced in the requirements. The results suggest that domain models should be complemented by other techniques for completeness checking. Future work should focus on user studies to evaluate whether analysts can effectively leverage domain models in practice.



In order to streamline the analysis we are going to replace the keys in both section dictionaries with simplified theme headings:

In [70]:
# simplified theme headings
themes = [
    'Introduction',
    'Methodology',
    'Findings',
    'Conclusion']

Swapping keys for `my_sections`:

In [71]:
# Create a mapping from old keys to new keys
mapping = dict(zip(my_themes, themes))

# Use a dictionary comprehension to create a new dictionary with swapped keys
# mapping.get(key, key) makes sure if a key is not in the mapping then there is no change
my_sections = {mapping.get(key, key): value for key, value in my_sections.items()}

In [72]:
print(my_sections['Conclusion'])

While domain models can clearly highlight missing requirements, this study did not evaluate whether analysts effectively identify and correct those omissions in practice. Future research should include user studies to explore the practical effectiveness of domain models in supporting requirements validation.

Conclusion
This empirical study provides concrete evidence supporting domain models' value as effective tools for completeness checking in natural-language requirements specifications. By systematically highlighting omissions, particularly entirely missing requirements, domain models can significantly improve requirements quality, making them valuable components of requirements engineering practice.



Swapping keys for `llm_sections`:

In [73]:
# Create a mapping from old keys to new keys
mapping = dict(zip(llm_themes, themes))

# Use a dictionary comprehension to create a new dictionary with swapped keys
# mapping.get(key, key) makes sure if a key is not in the mapping then there is no change
llm_sections = {mapping.get(key, key): value for key, value in llm_sections.items()}

In [74]:
print(llm_sections['Conclusion'])

The study provides empirical evidence that domain models can help identify missing and under-specified requirements, though their effectiveness depends on how frequently concepts are referenced in the requirements. The results suggest that domain models should be complemented by other techniques for completeness checking. Future work should focus on user studies to evaluate whether analysts can effectively leverage domain models in practice.

